# Sack Patrol
Because of scarcity marketing by microbrands, I must resort to this. I hope I learn my lesson.
This is set up specifically for sites using BigCartel.

In [1]:
!jt -t oceans16

%load_ext dotenv
%dotenv
# .env template
# TWILIO_SID=
# TWILIO_TOKEN=
# TWILIO_TO=
# TWILIO_FROM=
# POLL_URL=

import os
import requests
import polling2
import logging
import webbrowser
import time
import datetime  

from IPython.display import display, clear_output
import ipywidgets as widgets

from IPython.display import Audio
sound_file = './sound/iseeyou.wav'

from twilio.rest import Client

output = widgets.HTML()
CHECK = 0
ERROR = 0
def increment():
    global CHECK
    CHECK += 1
    
def error():
    global ERROR
    ERROR += 1

account_sid = os.getenv("TWILIO_SID")
auth_token  = os.getenv("TWILIO_TOKEN")
client = Client(account_sid, auth_token)
    
URL = os.getenv("POLL_URL")
START = datetime.datetime.now().strftime("%H:%M:%S")

In [2]:
## Main Check Function
def check(r):
#     print(datetime.datetime.now(),r)
    clear_output()
    display(output)
    increment()
    output.value="<h3>Checking...</h3><p>Started at "+START+" and checked "+str(CHECK)+" times; and "+str(ERROR)+" errors</p><p>Last Checked:"+datetime.datetime.now().strftime("%H:%M:%S")+" - Status:" +r.json()["status"]+"</p>"
    if r.json()["status"] != "sold-out":
        output.value = output.value+"<h2>It's there! <a href='"+URL+"' target='_blank'>Get it!</a></h2>"
        message = client.messages.create(
            to=os.getenv("TWILIO_TO"), 
            from_=os.getenv("TWILIO_FROM"),
            body=URL)
        webbrowser.open(URL, new=1)
        display(Audio(sound_file, autoplay=True))
        return True
    else:
        output.value = output.value+"<h2>Nope. Sold out.</h2>"
        return False

## Validate Text Message

In [3]:
message = client.messages.create(
    to=os.getenv("TWILIO_TO"), 
    from_=os.getenv("TWILIO_FROM"),
    body="Starting Sack Patrol")
print(message.sid)

SM245e753223424d96bbc8d898a68d4968


This is just a quick check to make sure that the success state functions.

## Main Loop
This is where the magic is. This will poll every 90s to see if the thing is not sold out according to the check function.
The loop breaks once it is back on sale, triggers a browser window to open, sends me a message via twillio, and plays a creepy audio file.

In [4]:

def poll():
    try:
        polling2.poll(
            lambda: requests.get(URL+'.js'),
            check_success=check,
            step=60*1,
            poll_forever=True,
            log=logging.DEBUG)
    except:   # or catch one specific error with 'except AttributeError:'
        error()
        time.sleep(60)
        poll()

In [ ]:
poll()

HTML(value='<h3>Checking...</h3><p>Started at 14:22:52 and checked 1812 times; and 2 errors</p><p>Last Checked…